In [1]:
import json
import pandas as pd

In [2]:
df = pd.read_csv('S2_answer.csv')

In [3]:
df.head()

,emotion,original_text,prompt,answer
0,disgust,she got a job and a new car then suddenly I be...,"disgust, she got a job and a new car then sudd...","Based on the given text, here are three styl..."
1,disgust,I just found out my husband is cheating on me....,"disgust, I just found out my husband is cheati...","Based on the given text, here are three poss..."
2,anger,So I came home to my dog chewing up my new fur...,"anger, So I came home to my dog chewing up my ...","Based on the given text, here are three styl..."
3,anger,Once I was waiting for a friend for an hour to...,"anger, Once I was waiting for a friend for an ...","Based on the given text, here are three poss..."
4,fear,The other morning at 4 am I was woken by my ne...,"fear, The other morning at 4 am I was woken by...","Based on the given text, here are three styl..."


In [4]:
# NaN 값을 포함하는 행 찾기
rows_with_nan = df[df.isna().any(axis=1)]
print(rows_with_nan)

Empty DataFrame
Columns: [emotion, original_text, prompt, answer]
Index: []


In [5]:
import re

def get_array (text) :
    # Find the index of '{' from the left
    start_index = text.find('[')

    # Find the index of '}' from the right
    end_index = text.rfind(']') + 1

    # Extract the array part
    array_part = text[start_index:end_index].strip() 

    return array_part

In [6]:
df['modifiers'] = df['answer'].apply(get_array)

In [7]:
df.head()

,emotion,original_text,prompt,answer,modifiers
0,disgust,she got a job and a new car then suddenly I be...,"disgust, she got a job and a new car then sudd...","Based on the given text, here are three styl...","[\n""Filmic Noir"",\n""Retro-Futurism"",\n""Surreal..."
1,disgust,I just found out my husband is cheating on me....,"disgust, I just found out my husband is cheati...","Based on the given text, here are three poss...","[\n""Vomit-like colors"",\n""Distorted shapes"",\n..."
2,anger,So I came home to my dog chewing up my new fur...,"anger, So I came home to my dog chewing up my ...","Based on the given text, here are three styl...","[[""High Contrast""], [""Bold Outlines""], [""Warm ..."
3,anger,Once I was waiting for a friend for an hour to...,"anger, Once I was waiting for a friend for an ...","Based on the given text, here are three poss...","[\n""Frustrated Gradients"",\n""Disruptive Textur..."
4,fear,The other morning at 4 am I was woken by my ne...,"fear, The other morning at 4 am I was woken by...","Based on the given text, here are three styl...","[[""Bold and High-Contrast Colors""], [""Warm and..."


In [8]:
string = df['modifiers'].iloc[0]
print(string)
string = string.replace("\\n", '')
print(string)
json_data = json.loads(string)
print(json_data)

[
"Filmic Noir",
"Retro-Futurism",
"Surreal Dreamscapes"
]
[
"Filmic Noir",
"Retro-Futurism",
"Surreal Dreamscapes"
]
['Filmic Noir', 'Retro-Futurism', 'Surreal Dreamscapes']


In [9]:
import re
def get_modifiers (string) :
    # 제거할 요소들
    elements_to_remove = ["modifiers", "artist", "color", "dimensionality", "style", "light", "modifier", "category", "perspective", "value", "typography", "type"]
    string = string.replace("\\n", '')
    try: 
        json_data = json.loads(string)
        values = [list(obj.values())[1] for obj in json_data]
        values = [item for item in values if item not in elements_to_remove]
        result_string = ", ".join(values)
        result_string = re.sub(r'(?i)typography', '', result_string)
        return result_string
    except Exception as e:
        print(f'org: {string}')
        
        # 제거할 요소들을 반복하여 문자열에서 삭제
        for element in elements_to_remove:
            string = string.replace('"' + element + '"', ' ')

        # 제거할 문자들
        characters_to_remove = r'[,:\[\]\{\}\\;]'

        # 정규 표현식을 사용하여 문자열에서 지정된 문자들을 모두 제거
        string = re.sub(characters_to_remove, '', string)

        # 정규 표현식을 사용하여 큰따옴표로 둘러싸인 부분을 추출
        extracted_list = re.findall(r'"(.*?)"', string)
        # 추출된 값을 필터링하여 요소 제거
        values = [item.strip() for item in extracted_list if item.lower().strip() not in elements_to_remove]

        result_string = ", ".join(values)
        result_string = re.sub(r'(?i)typography', '', result_string)

        print(result_string)

        return result_string

In [10]:
df['modifiers'] = df['modifiers'].apply(get_modifiers)

org: [
"Filmic Noir",
"Retro-Futurism",
"Surreal Dreamscapes"
]
Filmic Noir, Retro-Futurism, Surreal Dreamscapes
org: [
"Vomit-like colors",
"Distorted shapes",
"Grunge texture"
]
Vomit-like colors, Distorted shapes, Grunge texture
org: [["High Contrast"], ["Bold Outlines"], ["Warm Colors"]]
High Contrast, Bold Outlines, Warm Colors
org: [
"Frustrated Gradients",
"Disruptive Textures",
"Energized Highlights"
]
Frustrated Gradients, Disruptive Textures, Energized Highlights
org: [["Bold and High-Contrast Colors"], ["Warm and Cozy Tones"], ["Soft Glow and Ambient Lighting"]]
Bold and High-Contrast Colors, Warm and Cozy Tones, Soft Glow and Ambient Lighting
org: [ "Distorted Perspective", "Low Saturation", "Exaggerated Shading"]
Distorted Perspective, Low Saturation, Exaggerated Shading
org: [ "Angered", "Disputed", "Defiled" ]
Angered, Disputed, Defiled
org: [
"Bold, High-Contrast Colors",
"Distressed Textures",
"Exaggerated Shading"
]
Bold High-Contrast Colors, Distressed Textures, Exag

In [11]:
df.head()

,emotion,original_text,prompt,answer,modifiers
0,disgust,she got a job and a new car then suddenly I be...,"disgust, she got a job and a new car then sudd...","Based on the given text, here are three styl...","Filmic Noir, Retro-Futurism, Surreal Dreamscapes"
1,disgust,I just found out my husband is cheating on me....,"disgust, I just found out my husband is cheati...","Based on the given text, here are three poss...","Vomit-like colors, Distorted shapes, Grunge te..."
2,anger,So I came home to my dog chewing up my new fur...,"anger, So I came home to my dog chewing up my ...","Based on the given text, here are three styl...","High Contrast, Bold Outlines, Warm Colors"
3,anger,Once I was waiting for a friend for an hour to...,"anger, Once I was waiting for a friend for an ...","Based on the given text, here are three poss...","Frustrated Gradients, Disruptive Textures, Ene..."
4,fear,The other morning at 4 am I was woken by my ne...,"fear, The other morning at 4 am I was woken by...","Based on the given text, here are three styl...","Bold and High-Contrast Colors, Warm and Cozy T..."


In [12]:
df.to_csv('S2_modifiers.csv', index=False)